In [ ]:
# Requires brainvisa

In [1]:
import anatomist.api as anatomist

In [2]:
from soma import aims
import json
from soma.qt_gui.qt_backend import Qt
import sys
import pandas as pd
import scipy
import scipy.stats
import numpy as np

In [3]:
a = anatomist.Anatomist()

global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module meshsplit
loading module palettecontrols
loading module volumepalettes
loading module save_resampled
loading module profilewindow
loading module anacontrolmenu
loading module foldsplit
loading module simple_controls
loading module histogram
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


In [4]:
Rspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg"

json_regions = "/neurospin/dico/data/deep_folding/current/sulci_regions_gridsearch.json"

result_file = "/neurospin/dico/jchavas/flanker_per_region_on_HCP.ods" 

In [5]:
with open(json_regions) as f:
    regions = json.load(f)

In [6]:
next(iter(regions['brain']))

'S.C.-sylv._left'

In [7]:
res = pd.read_excel(result_file)
res = res[["Region (sulci_region_gridsearch)", "AUC", "std_auc"]].rename(columns = {"Region (sulci_region_gridsearch)": "region"})
res.head()

,region,AUC,std_auc
0,CINGULATE.,0.559,0.019
1,S.F.inf.-BROCA-S.Pe.C.inf.,0.494,0.023
2,Sc.Cal.-S.Li.,0.554,0.032
3,S.T.s.,0.516,0.041
4,S.T.s.br.,NaN,NaN


In [8]:
def compute_logp(auc, std_auc):
    p = scipy.stats.ttest_ind_from_stats(mean1=auc, 
                              std1=std_auc, 
                              nobs1=5, 
                              mean2=0.5, 
                              std2=0, 
                              nobs2=2, 
                              equal_var=False)[1]
    return -np.log10(p)

In [9]:
res["-log10_p"] =  res.apply(lambda x: compute_logp(x.AUC, x.std_auc), axis=1)
res

,region,AUC,std_auc,-log10_p
0,CINGULATE.,0.559,0.019,2.645969
1,S.F.inf.-BROCA-S.Pe.C.inf.,0.494,0.023,0.228425
2,Sc.Cal.-S.Li.,0.554,0.032,1.708908
3,S.T.s.,0.516,0.041,0.364390
4,S.T.s.br.,NaN,NaN,NaN
5,S.C.-sylv.,NaN,NaN,NaN
6,S.C.-S.Pe.C.,0.535,0.026,1.402889
7,S.C.-S.Po.C.,0.526,0.022,1.240918
8,S.Or.,0.486,0.060,0.201072
9,S.Or.-S.Olf.,NaN,NaN,NaN


In [10]:
def get_sulci(region):
    list_sulci = list(regions['brain'][f"{region}_right"].keys())
    list_sulci = ["S.F.int._right" if x=="paracingular._right" else x for x in list_sulci]
    return list_sulci

In [11]:
get_sulci("CINGULATE.")

['S.F.int._right', 'F.C.M.ant._right']

In [12]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
res.head()

,region,AUC,std_auc,-log10_p,sulcus
0,CINGULATE.,0.559,0.019,2.645969,"[S.F.int._right, F.C.M.ant._right]"
1,S.F.inf.-BROCA-S.Pe.C.inf.,0.494,0.023,0.228425,"[S.F.inf._right, S.F.inf.ant._right, F.C.L.r.a..."
2,Sc.Cal.-S.Li.,0.554,0.032,1.708908,"[F.Cal.ant.-Sc.Cal._right, S.O.p._right, S.Li...."
3,S.T.s.,0.516,0.041,0.364390,[S.T.s._right]
4,S.T.s.br.,NaN,NaN,NaN,"[F.I.P.r.int.1_right, F.I.P.r.int.2_right, S.T..."


In [13]:
res = res.sort_values(by="AUC", ascending=True)
res = res.explode("sulcus")
res.head()

,region,AUC,std_auc,-log10_p,sulcus
8,S.Or.,0.486,0.060,0.201072,S.Or._right
1,S.F.inf.-BROCA-S.Pe.C.inf.,0.494,0.023,0.228425,S.F.inf._right
1,S.F.inf.-BROCA-S.Pe.C.inf.,0.494,0.023,0.228425,S.F.inf.ant._right
1,S.F.inf.-BROCA-S.Pe.C.inf.,0.494,0.023,0.228425,F.C.L.r.ant._right
1,S.F.inf.-BROCA-S.Pe.C.inf.,0.494,0.023,0.228425,F.C.L.r.asc._right


In [14]:
for _, row in res.iterrows():
    print(row.sulcus)

S.Or._right
S.F.inf._right
S.F.inf.ant._right
F.C.L.r.ant._right
F.C.L.r.asc._right
F.C.L.r.diag._right
S.Pe.C.inf._right
F.I.P._right
F.I.P.Po.C.inf._right
OCCIPITAL_right
F.C.L.p._right
F.C.L.r.sc.ant._right
F.C.L.r.sc.post._right
INSULA_right
F.C.L.a._right
S.T.s._right
S.C._right
S.C.sylvian._right
S.Po.C.sup._right
F.I.P.Po.C.inf._right
F.C.L.r.retroC.tr._right
S.F.inter._right
S.F.sup._right
S.s.P._right
S.Pa.int._right
S.C._right
S.C.sylvian._right
S.Pe.C.inf._right
S.Pe.C.inter._right
S.Pe.C.marginal._right
S.Pe.C.sup._right
S.Pe.C.median._right
F.Cal.ant.-Sc.Cal._right
S.O.p._right
S.Li.ant._right
S.Li.post._right
S.F.median._right
S.F.polaire.tr._right
S.F.sup._right
S.F.int._right
F.C.M.ant._right
F.C.M.post._right
S.C.LPC._right
S.p.C._right
S.F.int._right
S.R.inf._right
S.R.sup._right
S.F.marginal._right
S.F.orbitaire._right
S.F.inf.ant._right
F.I.P.r.int.1_right
F.I.P.r.int.2_right
S.T.s.ter.asc.ant._right
S.T.s.ter.asc.post._right
S.GSM._right
S.C._right
S.C.sylvian._rig

In [22]:
res[res.sulcus=="S.F.orbitaire._right"]

,region,AUC,std_auc,-log10_p,sulcus
13,S.F.marginal-S.F.inf.ant.,0.609,0.009,4.956462,S.F.orbitaire._right


In [16]:
g = aims.read(Rspam_model)
ag = a.toAObject(g)



for vertex in g.vertices():
    vertex['AUC'] = 0.

for _, row in res.iterrows():
    for vertex in g.vertices():
        vname = vertex.get('name')
        if vname == row.sulcus:
            if pd.notnull(row['AUC']):
                vertex['AUC'] = row.AUC

ag.setColorMode(ag.PropertyMap)
ag.setColorProperty('AUC')
ag.notifyObservers()

w = a.createWindow('3D')
w.addObjects(ag)


In [17]:
res = res.sort_values(by="-log10_p", ascending=True)

In [18]:
g2 = aims.read(Rspam_model)
ag2 = a.toAObject(g2)

for vertex in g2.vertices():
    vertex['-log10p'] = 0.

for _, row in res.iterrows():
    for vertex in g2.vertices():
        vname = vertex.get('name')
        if vname == row.sulcus:
            if pd.notnull(row['-log10_p']):
                vertex['-log10p'] = row['-log10_p']

ag2.setColorMode(ag2.PropertyMap)
ag2.setColorProperty('-log10p')
ag2.notifyObservers()

w2 = a.createWindow('3D')
w2.addObjects(ag2)
